In [ ]:
from dotenv import load_dotenv
import os
import chromadb
import asyncio

load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"

> Download Ollama: https://ollama.com/download

In [ ]:
pip install pdfminer.six langsmith langchain langchain_openai chromadb pypdf nest_asyncio

In [ ]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
import faiss

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

# get embedding model
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [8]:
from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Je aime le programmation.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-12T14:24:26.3421796Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6606706800, 'load_duration': 3382393100, 'prompt_eval_count': 45, 'prompt_eval_duration': 2552058300, 'eval_count': 7, 'eval_duration': 660035500, 'model_name': 'llama3.2'}, id='run--08abae3e-7188-4dfb-a575-ad7313c9d90f-0', usage_metadata={'input_tokens': 45, 'output_tokens': 7, 'total_tokens': 52})